# Using the Banana Command Line Interface

On top of the Arcana's Python API, Banana adds a command line interface to make it easier to navigate the extensive analyses implemented within it.

All commands start with `banana` followed by a sequence of commands and switches depending on what you would like to do. The best place to start is with the `help` switch to find out about the options available.

In [1]:
! banana help

usage: banana <command> [<args>]

Available commands:
    avail      List all available analysis classes within Banana and custom
               search paths
    menu       Display the data and parameter specifications for a given
               analysis class
    derive     Generate derivatives from a Banana Analysis class
    gen-ref    Generate reference data for unit-tests
    help       Show help for a particular command

Base banana command

positional arguments:
  command        The sub-command to run

optional arguments:
  -h, --help     show this help message and exit
  --version, -v  show program's version number and exit


In [2]:
! banana avail


The following Analysis classes are available:

    mri.BoldAnalysis         BOLD functional MRI contrast
    mri.DwiAnalysis          Diffusion-weighted MRI contrast
    mri.DwiAndT1wAnalysis    A DWI series with a T1-weighted contrast images
                             co-registered to it provide anatomical
                             constraints to the tractography and generate a
                             connectome
    mri.T1AndT2wAnalysis     A T1-weighted contrast with a T2-weighted
                             contrast co-registered to it to improve the
                             segmentation of the peel surface in Freesurfer
    mri.T1wAnalysis          T1-weighted MRI contrast
    mri.T2starAnalysis       T2*-weighted MRI contrast
    mri.T2wAnalysis          T2-weighted MRI contrast



In [9]:
! banana menu mri.DwiAnalysis


banana.analysis.mri.dwi.DwiAnalysis Menu 
-----------------------------------------

Inputs:
    series : dicom, nifti, nifti_gz, extended_nifti_gz, analyze, mrtrix_image
        The set of EPI volumes that make up the series
    template (default=MNI152_T1::mni_tmpl_resolution) :
                                                 dicom, nifti, nifti_gz,
                                                 extended_nifti_gz, analyze,
                                                 mrtrix_image
        Template used for non-linear registration
    template_brain (default=MNI152_T1:brain:mni_tmpl_resolution) :
                                                 dicom, nifti, nifti_gz,
                                                 extended_nifti_gz, analyze,
                                                 mrtrix_image
        Brain-extracted template used for non-linear registration
    template_mask (default=MNI152_T1:brain_mask:mni_tmpl_resolution) :
                                       

In [4]:
! banana help derive

usage: banana derive [-h] [--dataset ARG [ARG ...]]
                     [--output_dataset ARG [ARG ...]]
                     [--processor ARG [ARG ...]] [--environment TYPE]
                     [--input SPEC PATTERN] [--parameter NAME VALUE]
                     [--subject_ids ID [ID ...]] [--visit_ids ID [ID ...]]
                     [--scratch PATH] [--cache SPEC [SPEC ...]]
                     [--enforce_inputs] [--reprocess] [--email EMAIL]
                     [--logger LOGGER LEVEL] [--quiet] [--bids_task BIDS_TASK]
                     dataset_path analysis_class analysis_name derivatives
                     [derivatives ...]

Generate derivatives from a Banana Analysis class

positional arguments:
  dataset_path          Either the path to the dataset if of 'bids' or 'basic'
                        types, or the name of the project ID for 'xnat' type
  analysis_class        Name of the class to instantiate
  analysis_name         The name of the analysis to put the analys

In [10]:
! banana derive data/ds000114 mri.T1wAnalysis my_banana_analysis2 brain_mask --subject_ids 01 --visit_ids test --scratch work --proc single

WARNING - Skipping unrecognised file '/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/ds000114/CHANGES' in BIDS tree
WARNING - Skipping unrecognised file '/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/ds000114/CHANGES' in BIDS tree
WARNING - Skipping unrecognised file '/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/data/ds000114/CHANGES' in BIDS tree
INFO - Optional input fileset 'coreg_ref' was not provided to T1wAnalysis(name='my_banana_analysis2').
INFO - Optional input fileset 'coreg_ref' was not provided to T1wAnalysis(name='my_banana_analysis2').
INFO - Optional input fileset 'coreg_ref' was not provided to T1wAnalysis(name='my_banana_analysis2').
INFO - Optional input fileset 'coreg_ref_brain' was not provided to T1wAnalysis(name='my_banana_analysis2').
INFO - Optional input fileset 'coreg_ref_brain' was not provided to T1wAnalysi

INFO - Executed 'brain_extraction_per_session_source' node in 0.0 minutes
INFO - Executed 'brain_extraction_per_session_source' node in 0.0 minutes
INFO - Executed 'brain_extraction_per_session_source' node in 0.0 minutes
INFO - [Node] Finished "brain_extraction.brain_extraction.brain_extraction_per_session_source".
191114-09:58:15,461 nipype.workflow INFO:
	 [Node] Finished "brain_extraction.brain_extraction.brain_extraction_per_session_source".
INFO - [Node] Setting-up "brain_extraction.brain_extraction.brain_extraction_bet" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/work/brain_extraction/brain_extraction/_subject_id_01/_visit_id_test/brain_extraction_bet".
191114-09:58:15,462 nipype.workflow INFO:
	 [Node] Setting-up "brain_extraction.brain_extraction.brain_extraction_bet" in "/Users/tclose/Documents/Workshops/2019-11-15-N.A.B.-workshop/nipype_arcana_workshop/notebooks/work/work/brain_extraction/brain_extraction/_subject_id